In [2]:
!pip install --quiet git+https://github.com/huggingface/transformers.git@5c00918681d6b4027701eb46cea8f795da0d4064
!pip install --quiet sentencepiece==0.1.95

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 880 kB 5.6 MB/s 
     |████████████████████████████████| 182 kB 44.7 MB/s 
     |████████████████████████████████| 3.3 MB 39.6 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 


In [3]:
!pip install --quiet ipython-autotime
%load_ext autotime

     |████████████████████████████████| 1.6 MB 4.7 MB/s 
time: 690 µs (started: 2022-11-26 04:51:36 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
from transformers import T5ForConditionalGeneration,T5Tokenizer

question_model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/t5/model/')
question_tokenizer = T5Tokenizer.from_pretrained('/content/gdrive/My Drive/t5/tokenizer/')

Mounted at /content/gdrive
time: 2min 59s (started: 2022-11-26 04:51:36 +00:00)


In [5]:
def get_question(sentence,answer,mdl,tknizer):
  text = "context: {} answer: {}".format(sentence,answer)
  print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]

  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

context = "Donald Trump is an American media personality and businessman who served as the 45th president of the United States."
answer = "Donald Trump"

ques = get_question(context,answer,question_model,question_tokenizer)
print ("question: ",ques)

context: Donald Trump is an American media personality and businessman who served as the 45th president of the United States. answer: Donald Trump


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:965: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1830: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


question:  Who is the 45th president of the United States?
time: 2.9 s (started: 2022-11-26 04:54:36 +00:00)


**UI using Gradio**

In [6]:
!pip install --quiet gradio==2.5.1

     |████████████████████████████████| 981 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 50.6 MB/s 
     |████████████████████████████████| 213 kB 44.9 MB/s 
     |████████████████████████████████| 593 kB 38.9 MB/s 
     |████████████████████████████████| 4.1 MB 50.7 MB/s 
     |████████████████████████████████| 856 kB 44.6 MB/s 
time: 15.9 s (started: 2022-11-26 04:54:39 +00:00)


In [7]:
import gradio as gr

context = gr.inputs.Textbox(lines=5, placeholder="Enter paragraph/context here...")
answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
question = gr.outputs.Textbox( type="auto", label="Question")

def generate_question(context,answer):
  return get_question(context,answer,question_model,question_tokenizer)

iface = gr.Interface(
  fn=generate_question, 
  inputs=[context,answer], 
  outputs=question)
iface.launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://3ea69ffe04431439.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://3ea69ffe04431439.gradio.app')

time: 6.19 s (started: 2022-11-26 04:54:54 +00:00)
